# 1. Importar librerías

In [191]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.options.display.float_format = '{:.2f}'.format

# 2. Importar base de datos

In [192]:
df = pd.read_csv('datasets/data_limpio.csv')

In [178]:
df.head()

,tipo_entidad,fecha_corte,unicap,desc_unicap,renglon,desc_renglon,saldo_a_la_fecha,vigente,v1_2,v2_3,...,riesgo_a_c,riesgo_a_s,riesgo_b_c,riesgo_b_s,riesgo_c_c,riesgo_c_s,riesgo_d_c,riesgo_d_s,riesgo_e_c,riesgo_e_s
0,1,2015-01-31,1,CRÉDITO ROTATIVO,5,CRÉDITO ROTATIVO TOTAL,37651351298.00,34981440061.00,768061743.00,579918756.00,...,6176,32878868826.00,270,1835206170.00,142,975343138.00,326,1368695340.00,69,593093538.00
1,1,2015-01-31,1,CRÉDITO ROTATIVO,5,CRÉDITO ROTATIVO TOTAL,826852228377.00,782016598064.00,9271463391.00,8065650992.00,...,125128,772666404396.00,1840,11740703689.00,1486,9872854505.00,1854,14111818486.00,2469,18460447300.00
2,1,2015-01-31,1,CRÉDITO ROTATIVO,5,CRÉDITO ROTATIVO TOTAL,108451564795.00,104345665139.00,1359378501.00,845847453.00,...,9723,100426465880.00,324,2920771751.00,207,1936819277.00,165,1623502258.00,144,1544005629.00
3,1,2015-01-31,1,CRÉDITO ROTATIVO,5,CRÉDITO ROTATIVO TOTAL,209307777744.00,196907738953.00,6328341960.00,2236769512.00,...,91873,196131426784.00,4802,6541005114.00,2385,2574770601.00,2515,3677977262.00,160,382597983.00
4,1,2015-01-31,1,CRÉDITO ROTATIVO,5,CRÉDITO ROTATIVO TOTAL,243818014709.00,227311255658.00,6333845197.00,2691829784.00,...,32098,224904584796.00,951,5969178631.00,848,5034273159.00,521,3607315495.00,520,4302662628.00


In [193]:
df['tipo_entidad'] = df['tipo_entidad'].astype('object')
df['unicap'] = df['unicap'].astype('object')
df['renglon'] = df['renglon'].astype('object')
df['fecha_corte'] = pd.to_datetime(df['fecha_corte'])

# 3. Correlaciones variables numéricas

In [194]:
cols_num = df.select_dtypes(exclude=['object', 'datetime'])
df_corr = cols_num.corr()
fig = px.imshow(df_corr[(df_corr > 0.7) | (df_corr < -0.7)],  text_auto=True)
fig.update_layout(height=1000)
fig.show()

## ¿Por qué el saldo a la fecha y el saldo vigente presentan una correlación tan alta?

El saldo a la fecha y el saldo vigente presentan una alta correlación. El saldo vigente representa el saldo luego de los abonos de los clientes a cada una de las cuentas en cada mes. Por tanto, se decide descartar el saldo vigente ya que saldo a la fecha y los abonos nos brindan la misma información.

In [195]:
df.drop('vigente', axis=1, inplace=True)

## ¿Qué hacer con las altas correlaciones en los rangos de vencimiento?

Las demás columnas, aunque tengan una alta correlación, se decide conservarlas porque representan características diferentes de las cuentas. Mes a mes cambia el tiempo que lleva una cuenta vencida. Algunos rangos de vencimiento se traslapan, esto puede ser porque cada entidad maneja diferentes estrategias de cobranza dependiendo del tipo de producto y por tanto estipula diferentes rangos para establecer el tiempo que lleva una cuenta vencida.

Una entidad, dependiendo del riesgo, para un producto implementa una estrategia para cuentas de 1 a 3 meses vencidas, mientras que para otro producto aplica una estrategia si la cuenta tiene entre 1 y 2 meses vencida y otra estrategia si la cuenta tiene entre 2 y 3 meses vencida.

In [196]:
cols_mora = df.iloc[:,8:21].columns
df.groupby(['tipo_entidad', 'desc_unicap'])[cols_mora].sum().iloc[1,:].to_frame()

,1
,CARTERA COMERCIAL CORPORATIVO
v2_3,0.00
v1_3,30243146591518.68
v3_4,0.00
v+4,0.00
v3_6,33059046066284.94
v+6,0.00
v1_4,0.00
v4_6,0.00
v6_12,43874611086107.93


Por ejemplo, una entidad de tipo 1, para carteras comerciales corporativas clasifica las cuentas cuentas vencidad de 1 a 3 meses, de 3 a 6 meses, de 6 a 12 meses y más de 12 meses.

In [197]:
df.groupby(['tipo_entidad', 'desc_unicap'])[cols_mora].sum().iloc[10,:].to_frame()

,1
,CONSUMO BAJO MONTO
v2_3,201552646764.76
v1_3,0.00
v3_4,0.00
v+4,0.00
v3_6,424888748982.12
v+6,132232049561.69
v1_4,0.00
v4_6,0.00
v6_12,0.00


Mientras que la misma entidad, para cuentas de consumos de bajo monto, las clasifica de 1 a 2 meses, de 2 a 3 meses, de 3 a 6 meses y más de 6 meses.

Decidimos trabajar con todas las variables y observar los resultados obtenidos con reducción de dimensionalidad.

## ¿Qué hacer con las correlaciones altas en algunos riesgos?

In [198]:
df['year_month'] = pd.to_datetime(df.fecha_corte.apply(lambda x: str(x.year) + '-' + str(x.month) + '-' + '01'))
base = df.groupby(['year_month'])[['saldo_a_la_fecha','riesgo_a_s','riesgo_b_s','riesgo_c_s','riesgo_d_s','riesgo_e_s']].mean()
fig = px.line(base)
fig.show()

In [199]:
base2 = df.groupby(['year_month'])[['riesgo_a_c','riesgo_b_c','riesgo_c_c','riesgo_d_c','riesgo_e_c']].mean()
fig = px.line(base2)
fig.show()

La gran mayoría de los clientes tienen obligaciones catalogadas como de riesgo a, que es la categoría de menos riesgo para las entidades. Dado que las columnas saldo por riesgo representan cada una el saldo total de cuentas que hay de dichos riesgos y la gran mayoría son de riesgos a, hay una correlación casi de 100% entre el saldo de cuentas de riesgo a y el saldo total a la fecha, se puede observar que varían casi que igual en el tiempo. Dicho lo anterior, se decide conservar el saldo total a la fecha y descartar las columnas de saldo por riesgos. Las columnas de número de clientes por riesgos también representa muy bien la frecuencia e importancia de cada riesgos.

In [200]:
df.drop(['riesgo_a_s', 'riesgo_b_s', 'riesgo_c_s', 'riesgo_d_s', 'riesgo_e_s'], axis=1, inplace=True)

## ¿Qué hacer con las correlaciones altas de los riesgos con el número de clientes con más de 1 mes en mora?

In [201]:
base3 = df.groupby(['year_month'])[['clientes_mora+30d','riesgo_a_c','riesgo_b_c','riesgo_c_c','riesgo_d_c','riesgo_e_c']].mean()
fig = px.scatter_matrix(base3)
fig.update_layout(height=800)
fig.show()

Dado que esta variable presenta una alta correlación con otras 4 variables, se decide descartarla.

In [202]:
df.drop(['clientes_mora+30d', 'year_month'], axis=1, inplace=True)

## Observar correlación

In [203]:
cols_num = df.select_dtypes(exclude=['object', 'datetime'])
df_corr = cols_num.corr()
fig = px.imshow(df_corr,  text_auto=True)
fig.update_layout(height=1000)
fig.show()